<a href="https://colab.research.google.com/github/MJ-GINI/AiffelRepository/blob/main/%5B%E1%84%83%E1%85%A6%E1%84%8A%E1%85%A16%E1%84%80%E1%85%B5%5D%E1%84%83%E1%85%A6%E1%84%8B%E1%85%B5%E1%84%90%E1%85%A5%E1%84%90%E1%85%A9%E1%86%AB_M2_%E1%84%82%E1%85%A9%E1%84%83%E1%85%A1%E1%86%B7%E1%84%8F%E1%85%A6%E1%84%8B%E1%85%A5%E1%84%91%E1%85%A9%E1%84%8B%E1%85%B5%E1%86%AB%E1%84%90%E1%85%B3_251201_jooyeon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**모델링**

In [ ]:
import pandas as pd
import numpy as np
import time
import warnings
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.calibration import CalibratedClassifierCV

# Install catboost
!pip install catboost

# 모델 라이브러리
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, StackingClassifier
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

warnings.filterwarnings('ignore')

In [ ]:
# 데이터 분리 및 스케일링
# 'smoking' 타겟은 이미 preprocess 함수에서 분리되어 `y` 변수에 저장되어 있습니다.

# X_train과 y는 preprocess 함수에서 전처리된 데이터입니다.

# X_test의 컬럼을 X_train의 컬럼과 일치시킵니다. (누락된 컬럼은 0으로 채움)
train_cols = X_train.columns
X_test_aligned = X_test.reindex(columns=train_cols, fill_value=0)

# 선형 모델(Logistic, SVM)을 위해 스케일링
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
# Test셋은 Train셋의 Scaler로 변환 (정렬된 X_test 사용)
test_final_scaled = pd.DataFrame(scaler.transform(X_test_aligned), columns=X_test_aligned.columns)

# 검증셋 분리 (8:2)
X_train_split, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

print(f"3. 데이터 준비 완료! (Train: {X_train_split.shape}, Val: {X_val.shape})")
print("="*60)

3. 데이터 준비 완료! (Train: (127404, 98), Val: (31852, 98))


In [ ]:
# --------------------------------------------------------------------------------
# [Step 1] 개별 모델 돌려보기 (Base Models)
# --------------------------------------------------------------------------------
print("[Step 1] 개별 모델 학습 및 평가 시작")

# NOTE: X_train, y, X_test는 preprocess에서 생성되었고,
# X_train_split, X_val, y_train, y_val은 이전 셀에서 생성되었습니다.
# 이 셀에서는 이미 생성된 X_train_split, X_val, y_train, y_val을 사용합니다.

# XGBoost 호환성을 위해 컬럼 이름 정리
# XGBoost는 피처 이름에 [, ] 또는 <와 같은 특수 문자를 허용하지 않음
X_train_split.columns = X_train_split.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
X_val.columns = X_val.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

# SVM은 확률값(predict_proba)이 안 나오므로 CalibratedClassifierCV로 감싸야 함
svm_base = LinearSVC(random_state=42)
svm_calibrated = CalibratedClassifierCV(svm_base, method='sigmoid', cv=3)

models = {
    "LGBM": LGBMClassifier(n_estimators=1000, learning_rate=0.05, num_leaves=31, random_state=42, verbose=-1),
    "XGB": XGBClassifier(n_estimators=1000, learning_rate=0.05, max_depth=6, random_state=42, n_jobs=-1),
    "CatBoost": CatBoostClassifier(n_estimators=1000, learning_rate=0.05, depth=6, random_state=42, verbose=0),
    "RF": RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1),
    "ExtraTrees": ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1),
    "Logistic": LogisticRegression(max_iter=1000, random_state=42),
    "SVM(Calibrated)": svm_calibrated
}

fitted_models = {}
results = {}

print(f"{'Model Name':<20} | {'Accuracy':<10} | {'AUC':<10} | {'Time'}")
print("-" * 55)

for name, model in models.items():
    start = time.time()
    model.fit(X_train_split, y_train)

    pred = model.predict(X_val)
    # SVM 등 일부 모델은 predict_proba 지원 여부 확인 필요 (Calibrated는 지원함)
    if hasattr(model, "predict_proba"):
        proba = model.predict_proba(X_val)[:, 1]
    else:
        proba = [0] * len(pred) # 예외처리

    acc = accuracy_score(y_val, pred)
    auc = roc_auc_score(y_val, proba)
    elapsed = time.time() - start

    print(f"{name:<20} | {acc:.4f}     | {auc:.4f}     | {elapsed:.1f}s")

    fitted_models[name] = model
    results[name] = acc

[Step 1] 개별 모델 학습 및 평가 시작
Model Name           | Accuracy   | AUC        | Time
-------------------------------------------------------
LGBM                 | 0.7803     | 0.8628     | 33.8s
XGB                  | 0.7794     | 0.8635     | 36.9s
CatBoost             | 0.7793     | 0.8635     | 53.2s
RF                   | 0.7701     | 0.8528     | 92.2s
ExtraTrees           | 0.7669     | 0.8487     | 98.6s
Logistic             | 0.7571     | 0.8408     | 4.6s
SVM(Calibrated)      | 0.7572     | 0.8411     | 25.6s


In [ ]:
# # --------------------------------------------------------------------------------
# # [Step 2] 가중치 있는 소프트 보팅 (Weighted Soft Voting)
# # --------------------------------------------------------------------------------
# print("\n[Step 2] 가중치 보팅 (Weighted Soft Voting) 진행")

# # 1. Step 1에서 학습된 모델들 가져오기 (fitted_models 딕셔너리 활용)
# # 성능이 좋았던 모델 Top 3(XGB, Cat, LGBM)에 가중치 높게
# # 로지스틱이나 SVM은 성능은 낮지만 모델의 다양성(Diversity)을 위해 소량 섞음

# # 예측 확률(Probability) 모으기
# # fitted_models는 Step 1에서 이미 학습된 모델들이 들어있는 딕셔너리입니다.
# preds = {
#     "LGBM": fitted_models["LGBM"].predict_proba(X_val)[:, 1],
#     "XGB": fitted_models["XGB"].predict_proba(X_val)[:, 1],
#     "CatBoost": fitted_models["CatBoost"].predict_proba(X_val)[:, 1],
#     "RF": fitted_models["RF"].predict_proba(X_val)[:, 1],
#     "Logistic": fitted_models["Logistic"].predict_proba(X_val)[:, 1],
#     "SVM": fitted_models["SVM(Calibrated)"].predict_proba(X_val)[:, 1]
# }

# # 2. 가중치 설정
# # - 1군(0.862+): XGB, CatBoost, LGBM -> 가중치 4
# # - 2군(0.85+): RF -> 가중치 2
# # - 3군(0.83+): Logistic, SVM -> 가중치 1 (다양성 확보용)

# weights = {
#     "LGBM": 4,
#     "XGB": 4,
#     "CatBoost": 4,
#     "RF": 2,
#     "Logistic": 1,
#     "SVM": 1
# }

# # 3. 가중 평균 계산 (Ensemble)
# final_pred_proba = np.zeros(len(X_val))
# total_weight = sum(weights.values())

# for name, weight in weights.items():
#     final_pred_proba += preds[name] * weight

# final_pred_proba /= total_weight # 가중치 합으로 나누기

# # 4. 성능 평가 (AUC & Accuracy)
# voting_auc = roc_auc_score(y_val, final_pred_proba)
# voting_acc = accuracy_score(y_val, np.where(final_pred_proba >= 0.5, 1, 0))

# print(f"[Voting 결과] 가중치 {list(weights.values())}")
# print(f"   - ROC AUC : {voting_auc:.5f} (Step 1 최고점보다 상승했는지 확인)")
# print(f"   - Accuracy: {voting_acc:.5f}")


In [ ]:
# --------------------------------------------------------------------------------
# [Step 3] 스태킹 (Stacking)
# --------------------------------------------------------------------------------
print("\n[Step 3] 스태킹 (Stacking)")

# Base로 쓸 모델 선정 (성능 좋은 것 + 성격 다른 것 조합)
estimators = [
    ('lgbm', models['LGBM']),
    ('xgb', models['XGB']),
    ('cat', models['CatBoost']),
    ('rf', models['RF']),
    ('et', models['ExtraTrees'])
]

# 메타 모델 (최종 결정자): 과적합 방지를 위해 보통 LogisticRegression을 많이 씀
meta_model = LogisticRegression(random_state=42)

stack_model = StackingClassifier(
    estimators=estimators,
    final_estimator=meta_model,
    cv=2, # 내부 교차 검증 (시간 걸리면 줄이세요)
    n_jobs=-1
)

stack_model.fit(X_train_split, y_train)
stack_pred = stack_model.predict(X_val)
stack_acc = accuracy_score(y_val, stack_pred)

print(f"Stacking Accuracy = {stack_acc:.4f}")